# Python API-Powered Educational Dataset Builder

---

**Name:**  Miguel Angel Gonzalez Mandujano  

**Institution:** UABC 

**Date:** 10/07/25


---

## Project Goal

Build a Python application that collects educational data from a public API, processes it with Pandas, and exports a clean dataset suitable for AI applications.

**Your Task:** Create a working dataset builder in 80 minutes!

**What you'll deliver:**
- Working Python code that connects to an API
- Collected data from at least 50 records
- Cleaned and processed dataset using Pandas
- Exported CSV file with your educational dataset


## Step 1: Choose Your API and Set Up Environment

**Choose an API from these options:**
- **NewsAPI** (news.org) - News articles
- **arXiv API** - Academic papers  
- **Reddit API** - Discussion posts
- **Wikipedia API** - Educational content
- **Guardian API** - News and articles
- **OpenLibrary API** - Book data

**Your chosen API:** 
arXiv API

**Why this API is good for education:**
Provides free, direct access to high-quality academic metadata (titles, abstracts, authors, dates) on highly specialized topics, such as Plasmonics or Thermoplasmonics.

### Import your libraries and set up configuration


In [1]:
# Import the libraries you need
# Hint: requests, pandas, time, json, numpy

import numpy as np
import pandas as pd
import time, json, requests

# Extras popurri
#---------------------------------------
# For the step 3 parse: xml -> dictionary 
# Import to parse the XML response from arXiv (arXiv doesn't work with Json)
import xml.etree.ElementTree as ET

# For the step 5: create additional features:
# Date/time features
from datetime import datetime

## Step 2: Connect to Your API

Create a function to make API requests with proper error handling.


In [2]:
# Create your API connection function
# Include error handling with try/except
# Test it with a simple request

# --- Configuration ---
# arXiv API base URL for searching papers
ARXIV_BASE_URL = "http://export.arxiv.org/api/query?"
MAX_RESULTS = 5 # Set a small number for testing the connection
SEARCH_QUERY =  'all:"thermoplasmonics" OR all:"hyperthermia" OR all:"photothermal"' #thermoplasmonics OR hyperthermia OR photothermal #'all:"thermoplasmonics"'
#SEARCH_QUERY = 'all:"thermoplasmonics" AND all:"nanogels"'

def fetch_arxiv_data(search_query: str, max_results: int = 50, start_index: int = 0) -> str:
    """
    Fetches data from the arXiv API using a search query and handles errors.

    Args:
        search_query (str): The string to search for in arXiv.
        max_results (int): Maximum number of records to return.
        start_index (int): <--- ADDED: The starting index for the search.

    Returns:
        str: The raw XML response content, or an empty string on error.
    """
    params = {
        'search_query': search_query,
        # --- FIX: Use the dynamic parameter here ---
        'start': start_index, 
        'max_results': max_results
    }
    
    # --- API Request with Error Handling ---
    try:
        # Build the full URL and make the GET request
        response = requests.get(ARXIV_BASE_URL, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)

        # Return the raw text content (which is XML for arXiv)
        return response.text

    except requests.exceptions.RequestException as e:
        # Handle specific request-related errors (e.g., connection issue, timeout)
        print(f"An API request error occurred: {e}")
        return ""
    except Exception as e:
        # Handle unexpected errors
        print(f"An unexpected error occurred: {e}")
        return ""


In [3]:
# --- Test the connection with a simple request ---
print(f"Testing connection for: {SEARCH_QUERY}")
test_xml_response = fetch_arxiv_data(SEARCH_QUERY, max_results=MAX_RESULTS)

if test_xml_response:
    print(f"Connection successful. Received XML response (first {len(test_xml_response)} characters):\n{test_xml_response[:400]}...")
else:
    print("Connection test failed.")

Testing connection for: all:"thermoplasmonics" OR all:"hyperthermia" OR all:"photothermal"
Connection successful. Received XML response (first 14122 characters):
<?xml version="1.0" encoding="UTF-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <link href="http://arxiv.org/api/query?search_query%3Dall%3A%22thermoplasmonics%22%20OR%20all%3A%22hyperthermia%22%20OR%20all%3A%22photothermal%22%26id_list%3D%26start%3D0%26max_results%3D5" rel="self" type="application/atom+xml"/>
  <title type="html">ArXiv Query: search_query=all:"thermoplasmonics" OR all:"hypert...


## Step 3: Collect Your Data

Write code to collect at least 50 records from your API. Include rate limiting (time delays) to be respectful to the API.


In [4]:
# Write your data collection function
# Collect at least 50 records
# Add time.sleep() between requests
# Store data in a list

# Your code here

# ARXIV_BASE_URL, SEARCH_QUERY, fetch_arxiv_data (function)
MAX_RECORDS_TO_COLLECT = 50       # Target: at least 50 records
RECORDS_PER_REQUEST = 10          # Number of records to fetch per API call (for smaller batches)
SLEEP_TIME_SECONDS = 3            # Delay between API calls for rate limiting

all_paper_data = []
ATOM_NS = 'http://www.w3.org/2005/Atom' # Namespace required to parse arXiv XML

#-----------------------------------------------------------------------------------------------------------
# --- XML Parsing Logic  ---
def parse_arxiv_entry(entry: ET.Element) -> dict:
    """Parses a single XML 'entry' element into a dictionary, con manejo seguro de atributos."""
    ns = {'atom': ATOM_NS}
    
    # Manejo seguro para 'primary_category'
    primary_cat_element = entry.find('atom:primary_category', ns)
    primary_category_term = primary_cat_element.attrib.get('term', None) if primary_cat_element is not None else None
    
    # Extract core metadata fields from the XML entry
    data = {
        'id': entry.find('atom:id', ns).text.replace('http://arxiv.org/abs/', ''),
        'published': entry.find('atom:published', ns).text,
        'title': entry.find('atom:title', ns).text.strip(),
        'summary': entry.find('atom:summary', ns).text.strip(),
        'primary_category': primary_category_term, # Usamos la variable segura
        'authors': [author.find('atom:name', ns).text for author in entry.findall('atom:author', ns)]
    }
    return data

In [5]:
# --- Data Collection Loop (FIXED) ---
print(f"Starting data collection for query: {SEARCH_QUERY}")
for start_index in range(0, MAX_RECORDS_TO_COLLECT, RECORDS_PER_REQUEST):
    print(f"Fetching batch: {start_index + 1} to {start_index + RECORDS_PER_REQUEST}...")
    
    # 1. Llamar a la función de conexión (fetch_arxiv_data) - FIX APLICADO AQUÍ
    # El tercer argumento 'start_index' le dice a la API dónde empezar a buscar.
    xml_response = fetch_arxiv_data(
        SEARCH_QUERY, 
        max_results=RECORDS_PER_REQUEST,
        start_index=start_index  # <--- PASSING THE DYNAMIC START INDEX
    )
    
    if not xml_response:
        print("Received empty response. Stopping collection.")
        break
    
    try:
        # 2. Parsear el XML
        root = ET.fromstring(xml_response)
        entries = root.findall(f'{{{ATOM_NS}}}entry')
        
        # ... (rest of the parsing and storage logic remains the same) ...
        # ... (Your storage, limit check, and time.sleep() logic are already correct) ...

        if not entries:
            print("No more records found for this query. Stopping collection.")
            break
        
        # 3. Extraer y almacenar data
        for entry in entries:
            all_paper_data.append(parse_arxiv_entry(entry))
        
        # 4. Verificar el límite y aplicar la pausa
        if len(all_paper_data) >= MAX_RECORDS_TO_COLLECT:
            all_paper_data = all_paper_data[:MAX_RECORDS_TO_COLLECT] 
            break

        print(f"Collected {len(all_paper_data)} records. Applying rate limit...")
        time.sleep(SLEEP_TIME_SECONDS) # Implement rate limiting
        
    except ET.ParseError as e:
        print(f"Error parsing XML at index {start_index}: {e}. Stopping collection.")
        break

print("\n--- Collection Complete ---")
print(f"Final Total Records Collected: {len(all_paper_data)}")

# Variable to be used in the next step: raw_paper_data
raw_paper_data = all_paper_data

Starting data collection for query: all:"thermoplasmonics" OR all:"hyperthermia" OR all:"photothermal"
Fetching batch: 1 to 10...
Collected 10 records. Applying rate limit...
Fetching batch: 11 to 20...
Collected 20 records. Applying rate limit...
Fetching batch: 21 to 30...
Collected 30 records. Applying rate limit...
Fetching batch: 31 to 40...
Collected 40 records. Applying rate limit...
Fetching batch: 41 to 50...

--- Collection Complete ---
Final Total Records Collected: 50


## Step 4: Process Your Data with Pandas

Convert your collected data to a DataFrame and clean it up.


In [6]:
# Convert your data to a pandas DataFrame
# Check for missing values
# Clean any messy text data
# Check the shape and basic info

# 1. Convert the list of dictionaries (raw_paper_data) into a DataFrame
df = pd.DataFrame(raw_paper_data)

# 2. Convert the 'published' field to a proper datetime object
# We only use 'published' as the 'updated' field was not included in our parser.
df['published_date'] = pd.to_datetime(df['published']).dt.tz_localize(None)

# 3. Clean the authors field 
# Convert the list of authors into a single, clean string for easier processing.
df['authors_str'] = df['authors'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# 4. Drop redundant columns 
# Remove the raw date string columns and the original list of authors.
# NOTE: We only drop 'published' now, as 'updated' did not exist.
df = df.drop(columns=['published', 'authors'])

# 5. Check the initial cleaning results and display basic info
print(f"DataFrame created with shape: {df.shape}")
print("\nData Types:")
print(df.dtypes)
print("\nMissing Values Check (Counts per column):")
print(df.isnull().sum())
print("\nFirst 3 rows of the processed DataFrame:")
print(df.head(3))

DataFrame created with shape: (50, 6)

Data Types:
id                          object
title                       object
summary                     object
primary_category            object
published_date      datetime64[ns]
authors_str                 object
dtype: object

Missing Values Check (Counts per column):
id                   0
title                0
summary              0
primary_category    50
published_date       0
authors_str          0
dtype: int64

First 3 rows of the processed DataFrame:
             id                                              title  \
0  2110.11932v1  Efficient Asymmetric photothermal source: Desi...   
1  2106.01059v4  Gold nanodoughnut as an outstanding nanoheater...   
2  2307.16165v1  Designing thermoplasmonic polymersomes for pho...   

                                             summary primary_category  \
0  Janus particles have flourished as subject of ...             None   
1  Photoinduced hyperthermia is a cancer therapy ...          

## Step 5: Create Additional Features

Add at least 3 new columns that make your dataset more useful for educational purposes.


In [7]:
# Create at least 3 new features, such as:
# - Text length
# - Word count  
# - Category classification
# - Quality score
# - Date/time features

# Your code here
# Step 5: Create Additional Features
# Add at least 3 new columns that make your dataset more useful for educational purposes.

# --- 1. Feature: Article Age (Years since publication) ---
# This feature is key for understanding the timeliness and relevance of the research.
# We use the published_date column created in Step 4.
current_year = datetime.now().year
df['published_year'] = df['published_date'].dt.year

# Calculate the age of the article in years
df['article_age_years'] = current_year - df['published_year']

# --- 2. Feature: Abstract Word Count ---
# This is a basic NLP feature, essential for models analyzing text density or length complexity.
# The 'summary' column holds the abstract text.
df['abstract_word_count'] = df['summary'].apply(lambda x: len(str(x).split()))

# --- 3. Feature: Collaboration Index (Number of Authors) ---
# A common research metric used to measure the scale of collaboration.
# We use the authors_str field we cleaned in Step 4 (authors separated by ', ').
df['author_count'] = df['authors_str'].apply(lambda x: len(x.split(', ')))

# --- Display the new features and final DataFrame structure ---
print(f"DataFrame shape after Feature Engineering: {df.shape}")
print("\nNew Features Added:")
print(df[['published_date', 'article_age_years', 'abstract_word_count', 'author_count']].head())

DataFrame shape after Feature Engineering: (50, 10)

New Features Added:
       published_date  article_age_years  abstract_word_count  author_count
0 2021-10-22 17:27:07                  4                  238             5
1 2021-06-02 10:25:27                  4                  173             8
2 2023-07-30 08:19:00                  2                  178             9
3 2024-08-27 06:08:16                  1                  134             2
4 2022-11-21 19:55:06                  3                  212             3


## Step 6: Export Your Dataset

Save your clean dataset as a CSV file and create basic documentation.


In [8]:
# Export your DataFrame to CSV
# Print a summary of what you created
# Show the first few rows of your final dataset

# Your code here

# Save your clean dataset as a CSV file and create basic documentation.

# --- 1. Final Cleanup: Drop the entirely missing column ---
# The primary_category column has 50 missing values and should be removed.
if 'primary_category' in df.columns:
    df = df.drop(columns=['primary_category'])

# --- 2. Export your DataFrame to CSV ---
# The project requires the CSV file to be named: LastName_FirstName_Module3_Dataset.csv
# NOTE: Replace 'YourLastName' and 'YourFirstName' with your actual names
CSV_FILENAME = 'Gonzalez_Miguel_Module3_Dataset.csv' 
df.to_csv(CSV_FILENAME, index=False)

# --- 3. Print Final Summary ---
print(f"✅ Final Dataset Exported Successfully to: {CSV_FILENAME}")
print(f"Final DataFrame Columns: {list(df.columns)}")
print(f"First 3 rows of FINAL dataset (ready for submission):")
print(df.head(3))

✅ Final Dataset Exported Successfully to: Gonzalez_Miguel_Module3_Dataset.csv
Final DataFrame Columns: ['id', 'title', 'summary', 'published_date', 'authors_str', 'published_year', 'article_age_years', 'abstract_word_count', 'author_count']
First 3 rows of FINAL dataset (ready for submission):
             id                                              title  \
0  2110.11932v1  Efficient Asymmetric photothermal source: Desi...   
1  2106.01059v4  Gold nanodoughnut as an outstanding nanoheater...   
2  2307.16165v1  Designing thermoplasmonic polymersomes for pho...   

                                             summary      published_date  \
0  Janus particles have flourished as subject of ... 2021-10-22 17:27:07   
1  Photoinduced hyperthermia is a cancer therapy ... 2021-06-02 10:25:27   
2  Polymersomes, vesicles self-assembled from amp... 2023-07-30 08:19:00   

                                         authors_str  published_year  \
0  Javier González-Colsa, Alfredo Franco Pérez,

## Project Summary

**What you built:**
A Python application for collecting and structuring high-quality academic research metadata.
API Used: arXiv API (http://export.arxiv.org/api/query).

Total Records Collected: 50 research papers!

Dataset Features Created: 
1. article_age_years (Research Timeliness)
2. abstract_word_count (Text Density/NLP Feature)
3. author_count (Collaboration Index/Research Metric)

**How this dataset could be used for educational AI:**
This dataset is specifically designed for graduate-level research applications in the Nanotechnology/Plasmonics field. It is suitable for training a specialized RAG system (Retrieval-Augmented Generation) or a Literature Recommendation Engine that could help graduate students quickly identify the most recent and highly collaborative research on their specific thesis topics. The engineered features (article_age_years and author_count) would allow the AI model to rank papers not just by keyword, but by relevance and potential influence.

**Challenges you encountered:**
The main challenge was successfully troubleshooting the XML AttributeError: 'NoneType' during the data parsing step. This demonstrated that real-world APIs require robust, safe parsing logic (xml.etree.ElementTree) to handle missing or inconsistent fields (like the missing category) across different records.

**What you learned about API programming:**
I learned the critical difference between the generic HTTP request (requests) and the data format (XML for arXiv), necessitating the use of specialized parsing for accurate data extraction. Furthermore, I reinforced the importance of rate limiting (time.sleep()) and error handling (try/except) as crucial steps to ensure the application's robustness and ethical interaction with public services.

In [9]:
df

,id,title,summary,published_date,authors_str,published_year,article_age_years,abstract_word_count,author_count
0,2110.11932v1,Efficient Asymmetric photothermal source: Desi...,Janus particles have flourished as subject of ...,2021-10-22 17:27:07,"Javier González-Colsa, Alfredo Franco Pérez, F...",2021,4,238,5
1,2106.01059v4,Gold nanodoughnut as an outstanding nanoheater...,Photoinduced hyperthermia is a cancer therapy ...,2021-06-02 10:25:27,"Javier González-Colsa, Guillermo Serrera, José...",2021,4,173,8
2,2307.16165v1,Designing thermoplasmonic polymersomes for pho...,"Polymersomes, vesicles self-assembled from amp...",2023-07-30 08:19:00,"Valentino Barbieri, Javier González-Colsa, Dia...",2023,2,178,9
3,2408.14798v1,Nonlinear thermoplasmonics in graphene nanostr...,The linear electronic dispersion relation of g...,2024-08-27 06:08:16,"Line Jelver, Joel D. Cox",2024,1,134,2
4,2211.11832v1,Photothermal and Thermo-optical Effects in 3D ...,"Thermonanophotonics, i.e. the study of phototh...",2022-11-21 19:55:06,"Alfredo Naef, Ted V. Tsoulos, Giulia Tagliabue",2022,3,212,3
5,1312.4737v1,Low-power photothermal probing of single plasm...,We demonstrate the direct photothermal probing...,2013-12-17 12:08:26,"Silvan Schmid, Kaiyu Wu, Peter Emil Larsen, To...",2013,12,125,5
6,2104.01964v1,All-dielectric thermonanophotonics,Nanophotonics is an important branch of modern...,2021-04-05 15:35:56,"George P. Zograf, Mihail I. Petrov, Sergey V. ...",2021,4,134,4
7,1801.07465v1,Thermoplasmonic Effect of Surface Enhanced Inf...,The temperature increase and temperature gradi...,2018-01-23 10:06:48,"Andrea Mancini, Valeria Giliberti, Alessandro ...",2018,7,220,7
8,2506.04835v1,Thermoplasmonics of Gold-Core Silica-Shell Col...,"Core-shell nanoparticles, particularly those h...",2025-06-05 09:57:13,"Julien El Hajj, Gilles Ledoux, Samy Merabia",2025,0,225,3
9,2411.16191v2,Plasmonic Janus particles: A perspective on op...,The compositional asymmetry of Janus micro- an...,2024-11-25 08:43:47,"Alemayehu Nana Koya, Anastasiia Sapunova, Nage...",2024,1,116,11


In [10]:
# 1. Configurar Pandas para no truncar texto (para ver el summary completo)
pd.set_option('display.max_colwidth', None)

# 2. Mostrar las primeras 10 filas con las columnas de interés
df[['title', 'summary']]

,title,summary
0,Efficient Asymmetric photothermal source: Designing a heating\n Janus-Nanojet,"Janus particles have flourished as subject of intensive research due to their\nsynergetic properties and their promising use in different fields, especially\nin biomedicine. The combination of materials with radically different physical\nproperties in the same nanostructure gives rise to the so-called Janus effects,\nallowing phenomena of a contrasting nature to occur in the same architecture.\nIn particular, interesting advantages can be taken from a thermal Janus effect\nfor photoinduced hyperthermia cancer therapies. Such therapies still have\nlimitations associated to the heating control in terms of temperature stability\nand energy management. While previous studies have shown that some plasmonic\nsingle-material nanoparticles are somehow effective at killing cancer cells, it\nis necessary to investigate alternative plasmonic Janus nanoheaters to improve\nthe heating efficiency and thermal control, mainly because the widespread\nsingle-material nanoheaters are highly homogeneous sources of heat, which\nimplies that the surrounding biological medium is isotropically heated, equally\naffecting cancerous and healthy cells. A detailed thermoplasmonic study of the\nthermal Janus effect is still missing. Here we perform such study and\ndemonstrate that doughnut-based Janus nanoparticles exhibit an outstanding\nphotothermal control under practical illumination conditions, i.e., unpolarized\nlight. Furthermore, we present novel and effective Janus nanoparticle designs\nthat possess superior photothermal conversion features and unique directional\nheating capacity, being able to channel up to 91% of the total thermal energy\nonto a target. We discuss the implications of these innovative nanoparticles\nwith regards to thermoplasmonics hyperthermia cancer therapy."
1,Gold nanodoughnut as an outstanding nanoheater for photothermal\n applications,"Photoinduced hyperthermia is a cancer therapy technique that induces death to\ncancerous cells via heat generated by plasmonic nanoparticles. While previous\nstudies have shown that some nanoparticles can be effective at killing cancer\ncells under certain conditions, there is still a necessity (or the need) to\nimprove its heating efficiency. In this work, we perform a detailed\nthermoplasmonic study comparing the most effective nanoparticle geometries up\nto now with a doughnut-shaped nanoparticle, demonstrating that the latter\nexhibits a superior tunable photothermal response in practical illumination\nconditions, i.e., unpolarized light. Furthermore, we show that nanoparticle\nheating in fluidic environments, i.e., nanoparticles undergoing Brownian\nrotations, strongly depends on the particle orientation with respect to the\nillumination source. We conclude that the heating performance of nanodoughnuts\nis outstanding, with a temperature increment 35% higher than the second best\nnanoheater (nanodisks). Furthermore, nanodoughnuts feature a weak dependence on\norientation, being therefore, ideal candidates for photothermal therapy\napplications. Finally, we present a designing guide, covering a wide range of\ntoroid designs, which can help on its experimental implementation."
2,Designing thermoplasmonic polymersomes for photothermal therapy,"Polymersomes, vesicles self-assembled from amphiphilic polymers, are\npromising nanocarriers for the targeted intracellular delivery of therapeutics.\nIntegrating inorganic light-absorbing materials with plasmonic properties, such\nas gold, into their membrane by in situ synthesis is a stepping stone to enable\ntheir use for photothermal therapy. Yet, it still needs to be determined\nwhether the in situ synthesis of gold can produce polymersomes with\nthermoplasmonic properties without altering their morphology, stability, and\nnanocarrier functionality. Here we demonstrate that small gold nanoparticles\ncan be controllably nucleated within biocompatible b